# Load libraries and data

In [1]:
import re
import json
import numpy as np
import torch
import onnx
import onnxruntime as ort
import torch.nn as nn

# Functions & Classes

In [2]:
def tokenizer(text, vocab, max_len=150):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text) # Remove special characters
    tokens = text.split()
    encoded = [vocab.get(word, vocab["<UNK>"]) for word in tokens[:max_len]]
    encodec_padded = np.pad(encoded, (0, max_len - len(encoded)), constant_values=vocab["<PAD>"])[:max_len]
    return torch.tensor(encodec_padded).unsqueeze(0)


class ToxicClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_filters, kernel_size, dropout, num_classes):
        super().__init__()
        # Embedding layers
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        # CNN layer
        self.conv = nn.Conv1d(
            in_channels=embed_dim,
            out_channels=num_filters,
            kernel_size=kernel_size,
            padding=1)
        self.pool = nn.AdaptiveMaxPool1d(50) # This reduces the sequence length
        # GRU layer
        self.gru = nn.GRU(
            input_size=num_filters,
            hidden_size=hidden_dim,
            batch_first=True,
            bidirectional=True)
        # Fully connected layer
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim * 2, num_classes)

    def forward(self, x):
        x = self.embedding(x) # (batch_size, seq_len, embedding_dim)
        x = x.permute(0, 2, 1) # change shape for conv1d (batch_size, channels, seq_len)
        x = torch.relu(self.conv(x))
        x = self.pool(x)
        x = x.permute(0, 2, 1) # change shape back for GRU (batch_size, seq_len, channels)
        x, _ = self.gru(x)
        x = self.dropout(x[:, -1, :]) # take the last time step
        return self.fc(x)


# Load model

In [3]:
# Load config
config = json.load(open("../model/config_no_pretrained.json"))

# Load vocab
vocab = json.load(open("../model/vocab_no_pretrained.json"))

# Recreate the model with saved hyperparameters
model = ToxicClassifier(
    vocab_size=config["vocab_size"],
    embed_dim=config["embedding_dim"],
    hidden_dim=config["hidden_dim"],
    num_filters=config["num_filters"],
    kernel_size=config["kernel_size"],
    dropout=config["dropout"],
    num_classes=config["num_classes"]
)

# Using CPU
model.to("cpu")

# Load weights
model.load_state_dict(torch.load("../model/model_no_pretrained.pth"))
model.eval()  # Set to evaluation mode

ToxicClassifier(
  (embedding): Embedding(184223, 50)
  (conv): Conv1d(50, 128, kernel_size=(3,), stride=(1,), padding=(1,))
  (pool): AdaptiveMaxPool1d(output_size=50)
  (gru): GRU(128, 64, batch_first=True, bidirectional=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=128, out_features=6, bias=True)
)

# ONNX

In [4]:
# Dummy input tensor: config["max_len"] = 150
dummy_input = torch.randint(0, 10000, (1, config["max_len"]))  # (batch_size, sequence_length)

# Convert PyTorch model to ONNX
onnx_model_path = "../model/model.onnx"
torch.onnx.export(
    model,
    dummy_input,
    onnx_model_path,
    input_names=["input"],
    output_names=["output"],
    dynamic_axes={"input": {0: "batch_size"}, "output": {0: "batch_size"}})

# Load and check the ONNX model
onnx_model = onnx.load(onnx_model_path)
onnx.checker.check_model(onnx_model)

/home/daniele/Desktop/Projects/toxic_comment_clf/.venv_tox_clf/lib/python3.10/site-packages/torch/onnx/symbolic_opset9.py:4277: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with GRU can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn(


save vocab and used configuration

In [5]:
# Load config
json.dump(config, open("../model/config_onnx.json", "w"))

# Save vocab
json.dump(vocab, open("../model/vocab_onnx.json", "w"))

In [6]:
# Load the ONNX model
session = ort.InferenceSession(onnx_model_path)

In [7]:
text = "I hate you"
input = tokenizer(text, vocab, max_len=config["max_len"])
input = input.numpy()

In [8]:
def tokenizer_onnx(text, vocab, max_len=150):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text) # Remove special characters
    tokens = text.split()
    encoded = [vocab.get(word, vocab["<UNK>"]) for word in tokens[:max_len]]
    encodec_padded = np.pad(encoded, (0, max_len - len(encoded)), constant_values=vocab["<PAD>"])[:max_len]
    encodec_padded = encodec_padded.reshape(1, -1)
    return encodec_padded

In [9]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def predict_onnx(text, session):
    MAX_LEN = 150 # expected input length
    input = tokenizer_onnx(text, vocab, max_len=MAX_LEN)
    # Get ONNX model predictions
    output = session.run(None, {"input": input})[0]
    output = sigmoid(output)
    return output[0]



In [10]:
predict_onnx(
    "screw you\nwhy dont you stick it up your fucking ass than lick it out, block it i dont give a shit you fucking bastard, suck my fucking BALLLLLSSSSSSS!!!!!!!!!!!!!!!",
    session)

array([0.9953999 , 0.3964395 , 0.98704195, 0.06865328, 0.9231589 ,
       0.23441562], dtype=float32)

In [11]:
predict_onnx(
    "I hate you",
    session)

array([0.74529725, 0.02067623, 0.2101489 , 0.03142855, 0.2423118 ,
       0.04919871], dtype=float32)

In [12]:
predict_onnx(
    "I love you",
    session)

array([3.2368802e-02, 6.8845999e-05, 3.4508775e-03, 5.0511525e-04,
       3.5228976e-03, 9.8692975e-04], dtype=float32)

# Predict

In [13]:
def predict(text, vocab, model):
    input = tokenizer(text, vocab, max_len=config["max_len"])
    # Make prediction
    output = model(input)
    prediction = torch.sigmoid(output).detach().numpy()
    print(["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"])
    print(prediction[0])

In [14]:
predict(
    "screw you\nwhy dont you stick it up your fucking ass than lick it out, block it i dont give a shit you fucking bastard, suck my fucking BALLLLLSSSSSSS!!!!!!!!!!!!!!!",
    vocab,
    model
)

['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
[0.9953999  0.39643934 0.98704195 0.06865328 0.92315876 0.23441555]


In [15]:
predict(
    "I hate you",
    vocab,
    model
)

['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
[0.74529725 0.02067623 0.21014903 0.03142856 0.24231187 0.04919871]


In [16]:
predict(
    "I love you",
    vocab,
    model
)

['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
[3.2368802e-02 6.8845933e-05 3.4508808e-03 5.0511438e-04 3.5228913e-03
 9.8692975e-04]


# Estimate Memory Usage in RAM

The approximate memory usage of the model when loaded is checked by counting the number of parameters and their data type

In [17]:
def get_model_memory(onnx_model_path):
    model = onnx.load(onnx_model_path)
    total_params = 0
    total_size = 0

    for tensor in model.graph.initializer:
        param_size = np.prod(tensor.dims) * np.dtype(onnx.helper.tensor_dtype_to_np_dtype(tensor.data_type)).itemsize
        total_params += np.prod(tensor.dims)
        total_size += param_size
    
    return total_params, total_size / (1024 * 1024)  # Convert to MB

params, memory = get_model_memory(onnx_model_path)
print(f"Total Parameters: {params}")
print(f"Estimated Memory Usage: {memory:.2f} MB")

Total Parameters: 9305748
Estimated Memory Usage: 35.50 MB
